### Set up spark context and SparkSession

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark RandomForest Regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/07 22:09:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.format('csv').\
                               options(header='true', \
                               inferschema='true').\
                               load("data/Advertising.csv",header=True);

df.show(5,True)
df.printSchema()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [3]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



### Convert the data to dense vector (features and label)

In [4]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
transformed=df.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



### Deal with the Categorical variables, even they are numeric, if a feature columns has no more than 4 distinct values, it will be considered categorical and will be indexed to improve training model.

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5,True)

+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



In [6]:
# Import LinearRegression class
from pyspark.ml.regression import RandomForestRegressor

# Define LinearRegression algorithm
rf = RandomForestRegressor() # featuresCol="indexedFeatures",numTrees=2, maxDepth=2, seed=42


### Split the data into training and test sets (40% held out for testing)

In [7]:
(trainingData, testData) = data.randomSplit([0.6, 0.4])

trainingData.show(5)
testData.show(5)

+---------------+-----+---------------+
|       features|label|indexedFeatures|
+---------------+-----+---------------+
| [5.4,29.9,9.4]|  5.3| [5.4,29.9,9.4]|
|[7.3,28.1,41.4]|  5.5|[7.3,28.1,41.4]|
|[7.8,38.9,50.6]|  6.6|[7.8,38.9,50.6]|
| [8.4,27.2,2.1]|  5.7| [8.4,27.2,2.1]|
|  [8.6,2.1,1.0]|  4.8|  [8.6,2.1,1.0]|
+---------------+-----+---------------+
only showing top 5 rows

+----------------+-----+----------------+
|        features|label| indexedFeatures|
+----------------+-----+----------------+
|  [0.7,39.6,8.7]|  1.6|  [0.7,39.6,8.7]|
|  [4.1,11.6,5.7]|  3.2|  [4.1,11.6,5.7]|
| [13.1,0.4,25.6]|  5.3| [13.1,0.4,25.6]|
| [17.2,4.1,31.6]|  5.9| [17.2,4.1,31.6]|
|[17.9,37.6,21.6]|  8.0|[17.9,37.6,21.6]|
+----------------+-----+----------------+
only showing top 5 rows



### Fit RandomForest Regression Model

If you decide to use the indexedFeatures features, you need to add the parameter featuresCol="indexedFeatures".

Pipeline Architecture

In [8]:
pipeline = Pipeline(stages=[featureIndexer, rf])
model = pipeline.fit(trainingData)

### Make test predictions with testdata

In [9]:
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("features","label", "prediction").show(5)

+----------------+-----+-----------------+
|        features|label|       prediction|
+----------------+-----+-----------------+
|  [0.7,39.6,8.7]|  1.6|9.062589285714285|
|  [4.1,11.6,5.7]|  3.2|8.029440634754021|
| [13.1,0.4,25.6]|  5.3|7.922495633676666|
| [17.2,4.1,31.6]|  5.9|8.542096478909865|
|[17.9,37.6,21.6]|  8.0|11.26800595238095|
+----------------+-----+-----------------+
only showing top 5 rows



### Evaluation

In [10]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 2.28356


In [16]:
evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print("R Squared (R2) on test data = %g" % evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.821315


In [17]:
model.stages[-1].featureImportances

SparseVector(3, {0: 0.5158, 1: 0.3345, 2: 0.1497})

### Feature importances, there are 3 features, index 0 is the root had highest feature importance value, 0.5158, .... This means, advertising on TV is most important feature

In [19]:
model.stages[-1].featureImportances

SparseVector(3, {0: 0.5158, 1: 0.3345, 2: 0.1497})

### Show all decision trees in the random forest

In [22]:
model.stages[-1].trees

[DecisionTreeRegressionModel: uid=dtr_a937c0faf570, depth=5, numNodes=49, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_b654902e7be7, depth=5, numNodes=61, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_fab451d5d8b4, depth=5, numNodes=45, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_29d0491b6306, depth=5, numNodes=49, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_1be7e2a7ce82, depth=5, numNodes=45, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_fc14fe2a020e, depth=5, numNodes=55, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_7517151116b2, depth=5, numNodes=39, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_c2890f314f09, depth=5, numNodes=39, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_8830b3380f0d, depth=5, numNodes=49, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_a01b3fa8101a, depth=5, numNodes=53, numFeatures=3,
 DecisionTreeRegressionModel: uid=dtr_46e0aa863aa5, depth=5, numNodes=51, numFeatures=3,
 DecisionTreeRegressi

In [14]:
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print("R Squared (R2) on test data = %g" % rf_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.821315


In [15]:
rf_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="rmse")
print("(RMSE) on test data = %g" % rf_evaluator.evaluate(predictions))

(RMSE) on test data = 2.28356
